## Imports

In [ ]:
import sys
import pandas as pd

# Tableau API connection
import tableauserverclient as TSC

# SQL server connection
import pyodbc
from sqlalchemy import create_engine
from datetime import date

from helpers import helpers_tableauserver as ts
from helpers import helpers_general as hg

*Some additional settings*

In [ ]:
# to show warning only once
import warnings; warnings.filterwarnings(action='once')

# to avoid warnings (if necessary)
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# to print all the outputs in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Global parameters

***NB! change Global parameters below with your credentials!***

### Tableau Development environment parameters

In [ ]:
server_url_dev = 'https://your_dev__server.tableau.example.com'    
token_name_dev = 'token_tableau_all_sites_dev'
token_value_dev = 'paste_token_value_dev'
site_name_dev = 'Default'  # any site name could be selected to start

### Tableau Production environment parameters

In [ ]:
server_url_prod = 'https://your_prod__server.tableau.example.com'    
token_name_prod = 'token_tableau_all_sites_prod'
token_value_prod = 'paste_token_value_prod'
site_name_prod = 'Default'  # any site name could be selected to start

### Parameters to store data from Tableau servers

*The data will be stored in a directory called `tableau_server_data` in the current working directory in Excel format*

In [ ]:
file_path_excel_permissions = './tableau_server_data/tableau_permissions_per_project.xlsx'
sheet_name = 'permissions_per_project'

*Additionally, the data will be stored in a Microsoft SQL server database.*

In [ ]:
server_sql = 'server_sql'
database = 'TEMP' 
schema_ref = 'dbo'
username = 'database_username'  
password = 'database_password'
table_permissions = 'permissions_per_project'

## Tableau development server - Get user's and group's permissions

### Open connection

In [ ]:
server = ts.connect_to_server_site(server_url_dev, token_name_dev, token_value_dev, site_name_dev)

*Get all permissions per project*

In [ ]:
df_permissions_dev = ts.get_all_server_permissions(server)

*Adding some information to the DataFrame*

In [ ]:
df_permissions_dev['tableau_server_type'] = 'dev'
df_permissions_dev['tableau_server_url'] = server_url_dev

### Close Tableau development server connection

In [ ]:
ts.sign_out_from_server(server)

### Do some EDA

In [ ]:
df_permissions_dev.shape
df_permissions_dev.info()
df_permissions_dev.sample(10)

## Tableau production server - Get user's and group's permissions

In [ ]:
server = ts.connect_to_server_site(server_url_prod, token_name_prod, token_value_prod, site_name_prod)

*Get all permissions per project*

In [ ]:
df_permissions_prod = ts.get_all_server_permissions(server)

*Adding some information to the DataFrame*

In [ ]:
df_permissions_prod['tableau_server_type'] = 'prod'
df_permissions_prod['tableau_server_url'] = server_url_prod

### Close Tableau production server connection

In [ ]:
ts.sign_out_from_server(server)

### collect all user's & group's permissions from 2 servers

#### Appending DataFrames

In [ ]:
df_all_permissions = df_permissions_dev.append(df_permissions_prod, ignore_index=True)

In [ ]:
df_all_permissions['udate_date'] = date.today()

In [ ]:
df_all_permissions.sample(15)

### Store data to Excel and to SQL Server database

*Write data to Excel*

In [ ]:
hg.write_to_excel(df_all_permissions, file_path_excel_permissions, sheet_name)

*Store data to SQL table*

In [ ]:
conn_str = f'mssql+pyodbc://{username}:{password}@{server_sql}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)

In [ ]:
df_all_permissions.to_sql(table_permissions, con=engine, if_exists='replace', index=False, dtype={'user_permissions': types.JSON })

*Confirm that data in the database*

In [ ]:
df_all_permissions.shape

In [ ]:
%%time

query = f'SELECT COUNT (1) FROM {schema_ref}.{table_permissions}'
_tmp = pd.read_sql(query, engine)
_tmp

In [ ]:
query = f'SELECT TOP (5) * FROM {schema_ref}.{table_permissions}'
_tmp = pd.read_sql(query, engine)
_tmp